In [6]:
import numpy as np
import pandas as pd
import chess
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torch.nn.functional as F
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


# DATASET CLASS

In [7]:
import torch
from torch.utils.data import Dataset
import chess

class EvalOnlyChessDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe.values
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        fen, evaluation = self.data[idx]

        board_tensor = torch.zeros((20, 8, 8), dtype=torch.float32)
        board = chess.Board(fen)

        piece_map = {
            'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
            'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
        }

        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece:
                rank, file = 7 - square // 8, square % 8
                board_tensor[piece_map[piece.symbol()], rank, file] = 1

        board_tensor[12] = 1.0 if board.turn == chess.WHITE else -1.0               # IZMENJENO - BITNO
        board_tensor[13] = int(board.has_kingside_castling_rights(chess.WHITE))
        board_tensor[14] = int(board.has_queenside_castling_rights(chess.WHITE))
        board_tensor[15] = int(board.has_kingside_castling_rights(chess.BLACK))
        board_tensor[16] = int(board.has_queenside_castling_rights(chess.BLACK))
        board_tensor[17] = int(board.has_legal_en_passant())
        board_tensor[18] = board.halfmove_clock / 50.0
        board_tensor[19] = board.fullmove_number / 100.0

        eval_value = evaluation

        eval_value = max(min(eval_value, 10.0), -10.0)
        eval_value /= 10.0

        return board_tensor, torch.tensor(eval_value, dtype=torch.float32)


In [8]:
from torch.utils.data import DataLoader
import pandas as pd

train_df = pd.read_csv("train2.csv", encoding="utf-8")
val_df = pd.read_csv("val2.csv", encoding="utf-8")

train_dataset = EvalOnlyChessDataset(train_df)
val_dataset = EvalOnlyChessDataset(val_df)

BATCH_SIZE = 512

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

# ARCHITECTURE

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, channels, dropout=0.1):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(channels)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(channels)
        self.dropout = nn.Dropout2d(dropout)

    def forward(self, x):
        residual = x
        out = F.leaky_relu(self.bn1(self.conv1(x)))
        out = self.dropout(out)
        out = self.bn2(self.conv2(out))
        return F.leaky_relu(out + residual)

class EvalResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(20, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU()
        )

        self.resblock1 = nn.Sequential(
            ResidualBlock(64),
            ResidualBlock(64)
        )

        self.downsample = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU()
        )

        self.resblock2 = nn.Sequential(
            ResidualBlock(128),
            ResidualBlock(128)
        )

        self.global_pool = nn.AdaptiveAvgPool2d((1, 1)) 

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128, 128),
            nn.LeakyReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1),
            nn.Tanh()  # Output ∈ [-1, 1]
        )

    def forward(self, x):
        x = self.initial(x)
        x = self.resblock1(x)
        x = self.downsample(x)
        x = self.resblock2(x)
        x = self.global_pool(x)
        x = self.fc(x)
        return x.squeeze(1)


# TRAINING

In [10]:
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm

model = EvalResNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.MSELoss()
EPOCHS = 15
best_val_loss = float('inf')

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]", leave=False)
    
    for inputs, targets in train_bar:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        preds = model(inputs)

        loss = criterion(preds, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
        train_bar.set_postfix({'loss': loss.item()})

    avg_train_loss = train_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1} [Val]", leave=False)

    with torch.no_grad():
        for inputs, targets in val_bar:
            inputs, targets = inputs.to(device), targets.to(device)
            preds = model(inputs)
            loss = criterion(preds, targets)
            val_loss += loss.item() * inputs.size(0)
            val_bar.set_postfix({'val_loss': loss.item()})

    avg_val_loss = val_loss / len(val_loader.dataset)

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), "eval_only_bestMSE.pth")
        print(f"Sacuvao novi model u epohi {epoch+1} sa val loss-om od {avg_val_loss:.4f}")

    print(f"\nEpoha {epoch+1} gotova.")
    print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")


✅ Saved new best model at epoch 1 with val loss 0.0476

Epoch 1 completed.
Train Loss: 0.0696 | Val Loss: 0.0476



Epoch 2 completed.
Train Loss: 0.0486 | Val Loss: 0.0527



Epoch 3 completed.
Train Loss: 0.0452 | Val Loss: 0.0535



Epoch 4 completed.
Train Loss: 0.0431 | Val Loss: 0.0583



Epoch 5 completed.
Train Loss: 0.0416 | Val Loss: 0.0479


...

✅ Saved new best model at epoch 11 with val loss 0.0345

Epoch 11 completed.
Train Loss: 0.0372 | Val Loss: 0.0345



Epoch 12 completed.
Train Loss: 0.0369 | Val Loss: 0.0503



Epoch 13 completed.
Train Loss: 0.0367 | Val Loss: 0.0423


✅ Saved new best model at epoch 14 with val loss 0.0339

Epoch 14 completed.
Train Loss: 0.0365 | Val Loss: 0.0339


✅ Saved new best model at epoch 15 with val loss 0.0334

Epoch 15 completed.
Train Loss: 0.0362 | Val Loss: 0.0334


In [11]:
# PODACI ZA GRAFIKE

# L1 DATA - LOSI PODACI (kod za podatke) PA JE I MREZA BILA LOSA:

l1_train_loss = [0.0267, 0.0148, 0.0127, 0.0134, 0.0085, 0.0070]
l1_val_loss = [0.0128, 0.0065, 0.0006, 0.0002, 0.0230, 0.0025]



### MSE DATA:
mse_train_loss = [0.0696, 0.0486, 0.0431, 0.0416, 0.404, 0.395, 0.0386, 0.0380, 0.0376, 0.0372, 0.0369, 0.0367, 0.0365, 0.0362]
mse_val_loss = [0.0476, 0.0527, 0.0535, 0.0583, 0.0479, 0.0446, 0.0415, 0.0388, 0.0432, 0.0381, 0.0345, 0.0503, 0.0423, 0.0339, 0.334]
